<a href="https://colab.research.google.com/github/nd82914/KeibaProject/blob/master/scrape_netkeiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  """
  !apt-get update
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium
  """
  from selenium import webdriver
  from selenium.webdriver.chrome.options import Options
  from urllib.parse import urljoin
  from bs4 import BeautifulSoup
  from google.colab import files
  import re
  import csv
  import pandas as pd
  from datetime import datetime
  from datetime import datetime as dt
  from datetime import timedelta
  global url_date

  def Read_Initxt():
      # ini.txtファイルからスクレイピングの対象をdict化する
      #files.upload()
      #ini_path = "C:/Envs/test/ini.txt"    # ini ファイルのみソース内で指定が必要。フルパスがよい。
      try:    # ファイルが開けなかったらエラーになるので、try~except 文を使いましょう。
          with open("ini_4.txt", 'r') as ini:
          #print(ini.read())
          #ini = open(ini_path, "r")    # txtファイルを読み取りモード（”ｒ”）で開く　
            ini_line = ini.readlines()    # txt内容を1行ずつ区切って配列にするメソッド
            ini_data = {}     # この空のdictの中にデータを詰めていきます
          for line in ini_line:
              line = line.rstrip("\n") # 改行コードを取り除く
              line = line.split(" ") # iniファイル内は" "で分けているので、それぞれの要素に分けて配列にする
              for i in range(1,len(line)): #index[0]はdictのkeyになる
                  if i == 1 : 
                      ini_data[line[0]] = []# {"key" : ["value1", "value2"...]}の形にする
                  if line[i] != "" and line[i][0] != "#":
                      ini_data[line[0]].append(line[i])
          ini.close()
          #print(ini_data)
      except:
          print("There is no initial files.")
          exit() #ini.txtが読み込めないとこちらに入ってメッセージを出力してプログラムを終了
      return(ini_data)

  def Output(out):
      with open("KeibaData", "w", encoding='utf-8') as data:
      #  data.write("【出力内容】")
      #data = open(ini_data["out_path"][0], "a", encoding='utf-8')    # 今度は追記モード("a")で開きます。ファイルがなければ作成されます。
          writer = csv.writer(data)

          writer.writerow(out)    # データの書込み
          data.close()    # 開けたファイルは最後に閉めましょう
          #files.download("KeibaData")

  def MakeHeadTable(source):
      """
      in=nettokeibaのソース
      out=th, td
      thは以下のリスト['レースID', 'レース番号','レース名','障害','左・右','内・外','距離[m]','天候','芝・ダ：状態','発走時刻','年月','開催場所','レースクラス','その他']
      tdはそのデータ
      """
      raceID=raceNo=raceName=None
      ditail=[None,None,None,None]
      condition=[None,None,None,None]
      raceDate=[None,None,None,None]

      data_intro = source.find("div", class_="data_intro")#属性がclassの場合class_と表記することに注意
      raceNo=data_intro.find("dt").get_text().replace(' ','').replace('\n','')
      raceName=data_intro.find("h1").get_text()
      condition=data_intro.find("span").get_text().replace(' ','').replace('\xa0\xa0',',').replace('\xa0/\xa0',' ').split()
      raceDate=data_intro.find("p", class_="smalltxt").get_text().split()
      #2020年3月29日 2回阪神2日目 3歳未勝利  [指](馬齢)　空白で分割しリスト化　4個以上なら4個になるまで[-1] を[-2]に結合
      if len(raceDate) < 4:
          print("Error:レース情報が足りません")
          exit
      elif len(raceDate)>4:
          print("Error:情報過多です。超過分をして出力します")
          while len(raceDate) > 4:
              raceDate[-2]=raceDate[-2]+raceDate[-1]
              del raceDate[-1]
      #['障害','右左','外内','距離']
      isSyougai = ('障' in condition[0])
      RightLeft = ('右' if '右' in condition[0] else '左' if '左' in condition[0] else False)
      OutIn = ('外' if '外' in condition[0] else '内' if '内' in condition[0] else False)
      distance=re.search(r'\d{3,}', condition[0]).group()#三桁以上の数字列を正規表現で抜き出す
      ditail=[isSyougai, RightLeft, OutIn, distance]

      csvRow=[]
      raceID=url_date
      csvRow.append(raceID)
      csvRow.append(raceNo)
      csvRow.append(raceName)
      csvRow.extend(ditail)
      csvRow.extend(condition[1:])
      csvRow.extend(raceDate)
      #print(csvRow)
      #ラップタイム,ペース追加
      raceRap=racePace=None
      RapPaceList=(raceRap,racePace)
      try:
          table_rap_pace = source.find("table",summary="ラップタイム")
          data_rap_pace=table_rap_pace.find_all("td")
          raceRap=data_rap_pace[0].get_text().replace(" ","").replace("\xa0","")
          racePace=data_rap_pace[1].get_text().replace(" ","").replace("\xa0","")
      except:
          pass
      csvRow.append(raceRap)
      csvRow.append(racePace)
      th=['レースID', 'レース番号','レース名','障害','左・右','内・外','距離[m]','天候','芝・ダ：状態','発走時刻','年月','開催場所','レースクラス','その他','ラップタイム','ペース']

      if len(csvRow) != len(th):
          print("Error:データタイトル数とデータ数が合いません")
          exit()
      else:
          pass
      return(th,csvRow)
  """
  def MakePosThTd(source):
      
      #in=nettokeibaのソース
      #out=th, td
      #thは以下のリスト['単勝No','単勝Odds','単勝Fav','複勝','複勝','複勝','枠連','枠連','枠連','年月','開催場所','レースクラス','その他']
      #tdはそのデータ
      
      th_sort=["単勝","複勝1","複勝2","複勝3","枠連","馬連","ワイド1","ワイド2ワイド3馬単三連複三連単]
      return(th,td)
      """

  def MakeHeadTable2(source):
      data_race_header = source.find("div",class_="Race_header")
      #raceNo取得
      try:
          raceDate=data_race_header.find("span",class_="Race_Date").get_text().replace("\n","")
      except:
          raceDate=None
      header_No_Place = []
      try:#raceNo,racePlace取得
          for header_tmp in data_race_header.find_all("option"):
              if "url_date" in header_tmp["value"]:#optionタグのリスト内に一致文字列があれば取り出す
                  header_No_Place.append(header_tmp.get_text())
          racePlace=header_No_Place[0]
          raceNo=header_No_Place[1]
      except:
          raceNo=None
          racePlace=None

      data_intro = source.find("div", class_="RaceHeader_Value")#属性がclassの場合class_と表記することに注意
      data_intro2 = data_intro.find_all("span")
      print(data_intro2)
      deleteNo=0
      raceName=raceTime=raceDitail=raceWhether=raceCondition=raceClass=raceEtc=None
      for introdata in data_intro2:
          print(introdata)
          try:
              if introdata["class"] == ["RaceName_main"]:
                  raceName = introdata.get_text()
          except:
              pass
          try:
              if "発走" in introdata.get_text():
                  raceTime = introdata.get_text()
          except:
              pass
          try:
              if introdata["class"] == ["Turf"]:
                  raceDitail = introdata.get_text()
          except:
              pass
          try:
              if introdata["class"] == ["WeatherData"]:
                  raceWhether = introdata.get_text()
          except:
              pass

      print(raceName,raceTime)

          
      try:
          raceTime=data_intro2[1].get_text()
      except:
          raceTime=None
          #print("raceTime=None")
      try:
          raceDitail=data_intro2[2].get_text()
      except:
          raceDitail=None
      try:
          raceWhether=data_intro2[3].get_text()
      except:
          raceWhether=None
      try:
          raceCondition=data_intro2[5].get_text()
      except:
          raceCondition=None
      try:
          raceClass=data_intro2[6].get_text()
      except:
          raceClass=None
      try:
          raceEtc=data_intro2[7].get_text()
      except:
          raceEtc=None

      #['障害','右左','外内','距離']
      isSyougai = ('障' in raceDitail)
      RightLeft = ('右' if '右' in raceDitail else '左' if '左' in raceDitail else None)
      OutIn = ('外' if '外' in raceDitail else '内' if '内' in raceDitail else None)
      distance=re.search(r'\d{3,}', raceDitail).group()#三桁以上の数字列を正規表現で抜き出す
      if '芝' in raceDitail and 'ダ' not in raceDitail:
          ShibaDa = '芝'
      elif 'ダ' in raceDitail and '芝' not in raceDitail:
          ShibaDa = 'ダ'
      elif '芝' in raceDitail and 'ダ' in raceDitail:
          ShibaDa = '芝・ダ'
      else:
          ShibaDa = None  
      ditail_list=[isSyougai, RightLeft, OutIn, distance, ShibaDa]

      td=[]
      td.append(raceNo)
      td.append(raceName)
      td.append(racePlace)
      td.extend(ditail_list)
      td.append(raceWhether)
      td.append(raceTime)
      td.append(raceCondition)
      td.append(raceDate)
      td.append(raceClass)
      td.append(raceEtc)
      th=['レース番号','レース名','場所','障害','左・右','内・外','距離[m]','芝・ダ','天候','発走時刻','馬場状態','年月','開催場所レースクラス','その他']
      if len(td) != len(th):
          print("Error:データタイトル数とデータ数が合いません")
          exit()
      return(th,td)
  def OutputHeadTable(th,td):
      print("--------------------------------headtable--------------------------------")
      print(th)
      print(td)
      data_name=url_date.replace("/","_")
      with open("HeadData%s.csv"%data_name, "w", encoding='utf-8') as data:
          writer = csv.writer(data)
          writer.writerow(th)
          writer.writerow(td)
      data.close()
      files.download("HeadData%s.csv"%data_name)
      print("--------------------------------headtable--------------------------------")
      !rm *.csv

  def OutputTable(Table_Souce):

      print("--------------------------------outputtable--------------------------------")
      rows = Table_Souce.find_all("tr")
      data_name=url_date.replace("/","_")
      with open("RaceData%s.csv"%data_name, "w", encoding='utf-8') as data:
          writer = csv.writer(data)
          i=0
          for row in rows:
              csvRow = []
              for cell in row.find_all(['td', 'th']):
                  #print(cell)
                  csvRow.append(cell.get_text().replace('\n',''))
              if i==0:
                csvRow=["レースID"] + csvRow
              else:
                csvRow=[url_date] + csvRow
              writer.writerow(csvRow)
              i+=1
              print(csvRow)
      data.close()
      files.download("RaceData%s.csv"%data_name)
      print("--------------------------------outputtable--------------------------------")
      !rm *.csv

  def pandalize(Table_Souce,preth,pretd):
      rows = Table_Souce.find_all("tr")
      i=0
      lists=[]
      for row in rows:
          csvRow = []
          for cell in row.find_all(['td', 'th']):
              #print(cell)
              csvRow.append(cell.get_text().replace('\n',''))
          if i==0:
            csvRow=preth + csvRow
            columns=csvRow
          else:
              csvRow=pretd + csvRow
              lists.append(csvRow)
          i+=1
      df = pd.DataFrame(data=lists,columns=columns)
      print(df.head())

  def Print_Soup():
      options = webdriver.ChromeOptions()
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument('--disable-dev-shm-usage')
      driver = webdriver.Chrome('chromedriver',options=options)
      driver.implicitly_wait(10)
      driver.get("https://www.netkeiba.com/")
      html = driver.page_source.encode('utf-8')
      soup = BeautifulSoup(html, "html.parser")
      print(soup.prettify())


  def Soup(url_link):
      #ini_data=Read_Initxt()
      #print(ini_data)
      # webサイトからデータを取得して、出力
      #try:
      result = []    # 書き込む内容を配列にする
      result.append(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))#　出力する時間を取得する
      # ブラウザのオプションを格納する変数をもらってきます。ブラウザをバックグラウンド起動するかなどを指定
      webdriver.ChromeOptions()
      #options = dr.chrome.options.Options()
      # options = dr.firefox.options.Options()    # Firefoxで行いたい場合はこちらを使います
      #options.set_headless(False)    # Trueにすると、バックグラウンド起動になります。
      options = webdriver.ChromeOptions()
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument('--disable-dev-shm-usage')
      
      # ブラウザを起動する
      #driver = dr.Chrome(executable_path=ini_data["driver_path"][0], chrome_options=options) # webdriver を立ち上げる
      driver = webdriver.Chrome('chromedriver',options=options)
      #driver.implicitly_wait(10)
      # driver = dr.Firefox(executable_path=ini_data["driver_path"][0], firefox_options=options) # Firefoxで行いたい場合はこちらを使います
      #link = ini_data["link"][0]
      link=url_link
      driver.get(link)# 指定したリンクのページにアクセスする
      html = driver.page_source.encode('utf-8')# アクセスしたページのhtmlソースを取得
      source = BeautifulSoup(html, "html.parser") # htmlソースの解析、lxmlではなぜか動かない
      th,td=MakeHeadTable(source)
      OutputHeadTable(th,td)
      OutputTable(source.find("table"))
      #pandalize(table[0],th,td)

      driver.close()

  def IterateDates():
      datelist = []
      dates=[]
      strdt = dt.strptime("20150401", '%Y%m%d')  # 開始日
      enddt = dt.strptime("20181219", '%Y%m%d')  # 終了日

      # 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
      days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります
      for i in range(days_num-1,-1,-1):
          datelist.append(strdt+timedelta(days=i))
      for date in datelist:
          dates.append(date.strftime('%Y%m%d'))
          #print(date.strftime('%Y%m%d'))
      print("開始日:%s"%strdt,"終了日:%s"%enddt,"日数:%d"%days_num)
      #print(dates)
      return(dates)

  def IterationScraping():
      global url_date #グローバル変数を関数内で変更するときは宣言が必要

      webdriver.ChromeOptions()
      options = webdriver.ChromeOptions()
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument('--disable-dev-shm-usage')
      
      # ブラウザを起動する
      driver = webdriver.Chrome('chromedriver',options=options)
      driver.implicitly_wait(5)
      # driver = dr.Firefox(executable_path=ini_data["driver_path"][0], firefox_options=options) # Firefoxで行いたい場合はこちらを使います
      #link = ini_data["link"][0]
      for date in IterateDates():
          link="https://db.netkeiba.com/race/list/%s/"%date
          print("access to link:%s"%link)
          driver.get(link)# 指定したリンクのページにアクセスする
          html = driver.page_source.encode('utf-8')# アクセスしたページのhtmlソースを取得
          source = BeautifulSoup(html, "html.parser") # htmlソースの解析、lxmlではなぜか動かない
          data_lists = source.find_all("div", class_="race_list fc")#属性がclassの場合class_と表記することに注
          if len(data_lists) == 0:
              print("%s:開催したレースがありません"%date)
              #driver.close()
              continue
          else:
              pass
          data_lists2 = source.find_all("a",href=re.compile("/race/\d+"))
          print("%s:開催レース数%d"%(date,len(data_lists)))
          for datalist in data_lists2:
              db_url=urljoin("https://db.netkeiba.com/",datalist.get("href"))
              url_date=datalist.get("href")#/race/202006030203/のような文字列をグローバル変数に代入
              print("URL:%s"%db_url)
              Soup(db_url)
      driver.close()


  #url_date="/race/201908040608/"
  #Soup("https://db.netkeiba.com/race/201908040608/")
  #Print_Soup()

  IterationScraping()
  #/race/202006030203/

In [ ]:
  !apt-get update
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium

In [ ]:
ls

In [ ]:
files.download("HeadData_race_201903020507_.csv")

In [ ]:
help(files)

In [ ]:
ls